# Imports
We import some neccesary libraries and download some global scope data

In [1]:
import re
import os
import json
import pandas as pd
import ast
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/nitram/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Load the Datsets

In [2]:

def merge_csv_files_from_folder(folder_path):
    # Lista para almacenar los DataFrames
    df_list = []

    # Recorrer todos los archivos en la carpeta
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            # Obtener la ruta completa del archivo
            file_path = os.path.join(folder_path, filename)
            
            # Leer el archivo CSV y agregarlo a la lista de DataFrames
            df = pd.read_csv(file_path)
            df_list.append(df)

    # Concatenar todos los DataFrames en uno solo
    if df_list:
        merged_df = pd.concat(df_list, ignore_index=True)
        return merged_df
    else:
        print("No se encontraron archivos CSV en la carpeta.")
        return pd.DataFrame()  

data = merge_csv_files_from_folder("data_processed/csv/")
data.drop(columns="car_id", inplace=True)
colnames = data.columns.to_list()
for col in colnames: print(col)

/tmp/ipykernel_3876/2912694450.py:12: DtypeWarning: Columns (40,41,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/tmp/ipykernel_3876/2912694450.py:12: DtypeWarning: Columns (40,41,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/tmp/ipykernel_3876/2912694450.py:12: DtypeWarning: Columns (40,41,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/tmp/ipykernel_3876/2912694450.py:12: DtypeWarning: Columns (40,41,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/tmp/ipykernel_3876/2912694450.py:12: DtypeWarning: Columns (40,41,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/tmp/ipykernel_3876/2912694450.py:12: DtypeWarning: Columns (40,41,44,45,46,47) 

price
km
year
color
cubicCapacity
brand
model
version
fuelType
bodyTypeId
transmissionTypeId
doors
seatingCapacity
warranty_months
province
environmentalLabel
co2Emissions
co2EmissionsGramsPerKm
trunkCapacityInLiters
maxSpeed
acceleration
hasDetails
jato_classification
traction
brakes
front_suspension
rear_suspension
power_cv
power_kw
rpm_max_power
max_torque_nm
rpm_max_torque
motor_description
transmission_description
speakers
trip_computer
remote_audio_control_on_steering_wheel
dvd_navigation
antenna
abs
electronic_traction_control
parking_sensors
airbag
stability_control
curve_braking_control
isofix_system
start_stop_automatic
cubicCapacity_seat_folded
Acabado Interior_Apoyabrazos
Acabado Interior_Asiento
Acabado Interior_Asientos
Acabado Interior_Dos
Acabado Interior_Pedales
Acabado Interior_Reloj
Acabado Interior_Reposacabezas
Acabado Interior_Tres
Acabado Interior_Uno
Confort_Aire
Confort_Apertura
Confort_Aviso
Confort_Banda
Confort_Calefacción
Confort_Cierre
Confort_Compartiment

In [3]:
for colum in [40,41,44,45,46,47]: data.iloc[:, colum] = data.iloc[:, colum].map({"True": True, "False": False})

/tmp/ipykernel_3876/2833469390.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
165432    NaN
165433    NaN
165434    NaN
165435    NaN
165436    NaN
Name: cubicCapacity_seat_folded, Length: 165437, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  for colum in [40,41,44,45,46,47]: data.iloc[:, colum] = data.iloc[:, colum].map({"True": True, "False": False})


In [4]:
def find_dirty_columns(prefix): return [col for col in data.columns if prefix in col]

# Cleaning Dirty Columns

There are four columns that require cleaning: `^Confort_`, `^Acabado Interior_` (already cleaned), `^Acabado Exterior_`, and `^electricFeatures`. You need to select one of these columns and process the data until it becomes readable. All columns should carry meaningful information. If a column contains a wide range of phrases, you can merge them into a single column labeled as "description," which will allow the application of NLP techniques later.

### Guidelines for Cleaning:

- **Singular and Plural Forms**:  
  If two columns represent the same feature, but one is the plural form of the other (e.g., `Confort_Luz` and `Confort_Luces`), discard the plural column.

- **Duplicate Column Names**:  
  If two columns share the same name, you can discard one of them.

- **Numbered Columns**:  
  If multiple columns are part of a series ending with a number (e.g., `Confort_Dos`, `Confort_Uno`), they likely describe different aspects of a single feature. In this case, merge them into a single column.

- **Relevance of Columns**:  
  Most of the dirty columns have names that describe the feature of their information, but some columns may be irrelevant.

- **Example for Reference**:  
  The column `^Acabado Interior_` has already been cleaned and can serve as an example of how to approach the cleaning process.

- **Nan and "no tiene"**:  
  Some data has an Nan and "no tiene", the first one means that we don't know the data but probably you can deduce that feature with other columns, specially in electrical features, and the second the car does nor have that feature.

**Data Types**:
  BE CAREFULL most of the cells that contains " ['Asiento de... " is a string not a list
  
At the beginning of each section, you will find a printout of the columns that need to be cleaned.

---

Let me know if you need further revisions or clarification!

## Functions

In [5]:
def remove_duplicates(lista, elemento): return [x for x in lista if x != elemento]

def check_unique_values(column):
    ret = sorted(set(data[column].to_list()))
    print(f"Column: {column}")
    for x in ret: print(f'{x}: {data[column].to_list().count(x)}')
    print(f'NA: {data[column].to_list().count(None)}')

## Clean Confort


In [38]:
confort_columns = find_dirty_columns("Confort")
for x in confort_columns: print(x)

Confort_Aire
Confort_Apertura
Confort_Aviso
Confort_Banda
Confort_Calefacción
Confort_Cierre
Confort_Compartimentos
Confort_Control
Confort_Dirección
Confort_Dos
Confort_Elevalunas
Confort_Faros
Confort_Indicador
Confort_Inmovilizador
Confort_Inmovilizador

Confort_Intermitentes
Confort_Lavafaros
Confort_Lavafaros

Confort_Limpiaparabrisas
Confort_Llave
Confort_Luces
Confort_Luz
Confort_Memoria
Confort_Protección
Confort_Regulación
Confort_Sistema
Confort_Tapa
Confort_Tarjeta
Confort_Techo
Confort_Testigo
Confort_Una
Confort_Volante


## Clean Acabado Interior 

In [39]:
acabado_interior_columns = find_dirty_columns("Acabado Interior")

In [40]:
acabado_interior_columns.remove("Acabado Interior_Asientos")
data.drop(columns="Acabado Interior_Asientos", inplace=True)

#for x in acabado_interior_columns: print(x)
#data[acabado_interior_columns]

In [41]:
data["sports_pedals"] = data["Acabado Interior_Pedales"].map({
    "['Pedales deportivos']": True, 
    'no tiene': False
})
data["interior_clock"] = data["Acabado Interior_Reloj"].map({
    "['Reloj analógico']": 'analogico', 
    "['Reloj digital']": 'digital',
    "['Reloj']": 'regular',
    "no tiene": 'no tiene'
})

#check_unique_values("sports_pedals")
#check_unique_values("interior_clock")


In [42]:

def process_arm_suppport_front(phrases: list) -> str:
    stop_words = set(stopwords.words('spanish'))
    stop_words.add(",")
    text = " # ".join(phrases)
    if not ('delantero' in text): return ("no tiene")
    text = [word for word in text.split() if word not in stop_words]
    if '#' in text: 
        ret = " ".join( text[ (text.index('delantero')+1):text.index('#')] )
        return "Tiene" if len(ret) == 0 else ret
    else: 
        ret = " ".join( text[(text.index('delantero')+1):len(text)] ) 
        return "Tiene" if len(ret) == 0 else ret

def process_arm_suppport_back(phrases: list) -> str:
    stop_words = set(stopwords.words('spanish'))
    stop_words.add(",")
    text = " # ".join(phrases)
    if not ('trasero' in text): return ("no tiene")

    text = [word for word in text.split() if word not in stop_words]

    ret = " ".join( text[(text.index('trasero')+1):len(text)] ) 
    return "Tiene" if len(ret) == 0 else ret 

data["arm_suppport_front"] = data["Acabado Interior_Apoyabrazos"].map(lambda x: x if x == 'no tiene' else process_arm_suppport_front(eval(x)) )
data["arm_suppport_back"] = data["Acabado Interior_Apoyabrazos"].map(lambda x: x if x == 'no tiene' else process_arm_suppport_back(eval(x)) )

check_unique_values("arm_suppport_front")
check_unique_values("arm_suppport_back")

Column: arm_suppport_front
Tiene: 117241
abatible: 98
ajustable: 1
caja integrada: 4238
no tiene: 43859
NA: 0
Column: arm_suppport_back
Tiene: 64993
abatible: 5
acceso maletero: 406
ajustable: 2
caja integrada: 1415
caja integrada acceso maletero: 559
no tiene: 98057
NA: 0


In [43]:
def process_seat_description(phrases: list) -> str: return ". ".join(phrases)

data["seat_description"] = data["Acabado Interior_Asiento"].map( lambda x: x if x == 'no tiene' else process_seat_description(eval(x)) )

In [44]:
data["Acabado Interior_Uno"] = data["Acabado Interior_Uno"].map( lambda x: "['no tiene']" if x == 'no tiene' else x )
data["Acabado Interior_Dos"] = data["Acabado Interior_Dos"].map( lambda x: "['no tiene']" if x == 'no tiene' else x )
data["Acabado Interior_Tres"] = data["Acabado Interior_Tres"].map( lambda x: "['no tiene']" if x == 'no tiene' else x )
data["Acabado Interior_Reposacabezas"] = data["Acabado Interior_Reposacabezas"].map( lambda x: "['no tiene']" if x == 'no tiene' else x )

data["Acabado Interior_Uno"] = data["Acabado Interior_Uno"].map(ast.literal_eval)
data["Acabado Interior_Dos"] = data["Acabado Interior_Dos"].map(ast.literal_eval)
data["Acabado Interior_Tres"] = data["Acabado Interior_Tres"].map(ast.literal_eval)
data["Acabado Interior_Reposacabezas"] = data["Acabado Interior_Reposacabezas"].map(ast.literal_eval)

In [45]:
data['head_supporter'] = data.apply(
    lambda row: ". ".join(
        remove_duplicates(row["Acabado Interior_Uno"] + row["Acabado Interior_Dos"] + row["Acabado Interior_Tres"] + row["Acabado Interior_Reposacabezas"], 'no tiene')
    ), 
    axis=1)
#check_unique_values("head_supporter")

for col in acabado_interior_columns: data.drop(columns= col, inplace=True)

## Clean Acabado Exterior

In [14]:
def cleanTry(phrases: list) -> str:
    stop_words = set(stopwords.words('spanish'))
    stop_words.add(",")
    text = "".join(phrases)

    text = [word for word in text.split() if word not in stop_words]

    ret = " ".join(text)
    return "no tiene" if len(ret) == 0 else ret


In [16]:
acabado_exterior_columns = find_dirty_columns("Acabado Exterior")
data.drop(columns=["Acabado Exterior_Maletero","Acabado Exterior_Pasos","Acabado Exterior_Maletero","Acabado Exterior_Defensa","Acabado Exterior_Guardabarros"], inplace=True)
for x in acabado_exterior_columns: 
    print(x)
    print("Descripcion --------------------------")
    print(data[x].describe())
    print("Counts--------------------------------")
    print(data[x].value_counts())
    print("CleanTry------------------------------")
    print(data[x].map( lambda x: x if x == 'no tiene' else cleanTry(x) ).value_counts())

Acabado Exterior_Alerón
Descripcion --------------------------
count       165437
unique          39
top       no tiene
freq         93676
Name: Acabado Exterior_Alerón, dtype: object
Counts--------------------------------
Acabado Exterior_Alerón
no tiene                                                                                                                                    93676
['Alerón en el techo/parte superior del portón']                                                                                            62666
['Alerón en el maletero/parte inferior del portón']                                                                                          5324
['Alerón en techo/parte superior del portón']                                                                                                2037
['Alerón en el techo/parte superior del portón en el maletero/parte inferior del portón']                                                     539
['Alerón en el maletero

## Electronic Features

In [47]:
print(data["fuelType"])
#check_unique_values("fuelType") Pone que hay un float en esa columna, solucionalo


0                   Gasolina
1                   Gasolina
2                   Gasolina
3                  Eléctrico
4                    Híbrido
                 ...        
165432                Diésel
165433              Gasolina
165434               Híbrido
165435    Híbrido enchufable
165436              Gasolina
Name: fuelType, Length: 165437, dtype: object


# Save file

In [48]:
max_file_size = 100 * 1024 * 1024
output_folder = "NLP/test_data"
def split_dataframe_to_csv(df, max_file_size, base_filename):
    file_index = 1
    chunk_size = len(df) // 7 
    while chunk_size > 0:
        start_row = 0
        while start_row < len(df):
            end_row = start_row + chunk_size
            chunk = df.iloc[start_row:end_row]
            filename = f"{output_folder}/{base_filename}_part{file_index}.csv"
            chunk.to_csv(filename, index=False)
            if os.path.getsize(filename) > max_file_size:
                chunk_size //= 2
                os.remove(filename)
            else:
                start_row = end_row
                file_index += 1

        if start_row >= len(df):
            break

split_dataframe_to_csv(data, max_file_size, base_filename="test")